# Logic for knowledge representation

This workbook explores the concept of **logical programming** in Python and how to use it for reasoning.

Throughout the notebook you will use two libraries, namely pytholog and miniKanren. You will find some guided examples to aid your understanding, and one exercise for you to implement on your own.

#### Content:
* [pytholog](#pl)
    * [Getting started](#pl-start)
* [miniKanren](#mk)
    * [Getting started](#mk-start)
    * [Example - The research-grant problem](#grant)
* [Exercise - Logic puzzles](#logic)

## pytholog <a class="anchor" id="pl"></a>

[Pytholog](https://mnoorfawi.github.io/pytholog/) is a Python library that provides tools for logic programming. It allows you to write logical statements and perform reasoning tasks using the [PROLOG](https://en.wikipedia.org/wiki/Prolog) programming language [syntax](https://en.wikipedia.org/wiki/Prolog_syntax_and_semantics).


**Facts**

Facts in Prolog are formed by a _predicate_ and some _terms_:
```prolog
parent(fran, robert).  
parent(robert, john).      
parent(john, paul).        
parent(john, ann).        
parent(paul, tom).          
parent(tom, mary).       
```
`parent(fran, robert)` (= Fran is Robert's parent) is a fact where parent is a predicate and (fran,robert) are terms. 

Note that  _terms in the facts are lowercased, meaning they are truths and cannot change._

**Rules**

Rules have a left-hand-side (LHS) containing a predicate, and a right-hand-side containing the _goals_ to match to answer a query:
```prolog
grandparent(X, Z) :- parent(X, Y), parent(Y, Z).
```
the rule above defines that X is granparent of Z (predicate), if X is parent of Y and Y is parent of Z (goals). 

In prolog: `,=AND` and `;=OR` 

Note that _terms in the rules are uppercased, meaning that they are variables, need to be changed in a query_.



**We are now going to implement the above facts and rules in pytholog.**

### Getting started <a class="anchor" id="pl-start"></a>

In [ ]:
# uncomment the cells below to install pytholog
#! pip install --upgrade pip
#! pip install pytholog

import pytholog as pl

In [ ]:
# let's start by defining a knowledge base object to store facts and  rules
kb = pl.KnowledgeBase('family_tree')

In [ ]:
# add facts and rules to kb
kb([
    'parent(fran, robert)',
    'parent(robert, john)',
    'parent(john, paul)',
    'parent(john, ann)',
    'parent(paul, tom)',
    'parent(tom, mary)',
    'grandparent(X, Z) :- parent(X, Y), parent(Y, Z)',
    'ancestor(X, Z) :- parent(X, Y),  grandparent(Y, Z)'
])

In [ ]:
# the following prints the knowledge stored into our KB
kb.db

We now want to query our KB to extract the following information:

* **q1**: who are John's children?
* **q2**: is John Fran's granparent?
* **q3**: who is Mary's ancestor?

In [ ]:
# Q1: who are John's children?
q1 = pl.Expr('parent(john, Y)')
kb.query(q1)

In [ ]:
# Q2: is John Fran's grandparent?
q2 = pl.Expr('grandparent(john, fran)')
kb.query(q2)

In [ ]:
# Q3: who is Mary's ancestor?
q3 = pl.Expr('ancestor(X, mary)')
kb.query(q3)

pytholog is a user-friendly and intuitive library that follows the PROLOG syntax. However, it may not perform optimally for complex problems, for example when there is need of mathematical comparison or inference with partial facts. 

In such cases, one option is to directly implement the problem using the PROLOG language. You can use the [SWISH online environment](https://swish.swi-prolog.org/example/prolog_tutorials.swinb) to practice coding in PROLOG. Give a look at these PROLOG [examples](https://swish.swi-prolog.org/example/examples.swinb) to get an idea of the syntax and capabilities of this language.


Alternatively, if you prefer to stick with Python, another solution is to explore [pySWIP](https://pypi.org/project/pyswip/), although it may require more effort to set up and work with since you will need a working instance of SWI-PROLOG in your laptop.
You can also look into [pylog](https://pypi.org/project/pylog/) and [pyke](https://pyke.sourceforge.net/), altough these libraries are not actively maintaned.

## miniKanren <a class="anchor" id="mk"></a>

[miniKanren](http://minikanren.org/) is family of languages for logic programming, implemented in a variaty of host languages, including Python.

The [miniKanren library](https://pypi.org/project/miniKanren/) for Python provides tools for logical programming. It allows you to write logical statements and perform reasoning tasks using a declarative approach. In contrast to pytholog, miniKanren takes a more familiar approach for Python developers as it is implemented purely in Python.

### Getting started <a class="anchor" id="mk-start"></a>

In [ ]:
# uncomment the cells below to install miniKanren
#! pip install --upgrade pip
#! pip install miniKanren

import kanren as mk

# We'll access these functions directly through mk using mk.<name_of_function>
# In an actual project, you might want to import them into python rather than through mk
# from kanren import fact, Relation, run, eq, membero, unifiable, vars, var, lall, lany
# from kanren.term import applyo

In `kanren` knowledge is stored using `facts` and relations among terms.

Let's consider the same family tree we implemented in pytholog:
- fran is robert's parent
- robert is john's parent
- john is paul and ann' parent
- paul is tom's parent
- tom is mary's parent

Here relation is _parent_ and terms are _fran, robert, joh, paul, tom, mary_.

So let's translate it using kanren.

In [ ]:
# let's start by defining the relation
parent = mk.Relation()

In [ ]:
# add facts and terms
mk.facts(parent, 
         ('Fran', 'Robert'),
         ('Robert', 'John'),
         ('John', 'Paul'),
         ('John', 'Ann'),
         ('Paul', 'Tom'),
         ('Tom', 'Mary')
)

The code above creates the knowledge base (KB) of our system.

We now want to query our KB to extract the following information:

* **q1**: who are John's children?
* **q2**: is John Fran's granparent?
* **q3**: who is Mary's ancestor?

In `kanren` we can run queries against a KB using the following syntax:

```
run(no_of_results, vars, goals(vars & terms))
```
- `no_of_results`: this is the number of results we want to get; use 0 for all.
- `vars`: unknown variables, used to query a KB with a question
- `goals`: these are logical statements, relations that need to be verified; they can contain unknowns variables for which we want an answer, or knows variables (= terms) to verify are goals agains to. **goals** allow you to specify conditions or relationships that you want to satisfy or find solutions for.

The function `run` return a tuple of results matching our goals/query.

**Q1: who are John's children?**

To answer this question our goal is to check for all the pairs in the KB of the type ('John', x), where x is the answer(s) we are looking for.

In [ ]:
# let's start by defining an unknown kanren variable
x = mk.var()

# let's query the KB
# We want to return all the results (0), our goals is to check for 'parent('John',x)', where x is unknown
results = mk.run(0, x, parent('John',x))

print('John`s children: ', results)

**Q2A: is John Fran's grandparent?**

To answer this questions we need to create a goal that defines what a 'grandparent' is. 

In kanren the _goal constructor_ is a typical python function.

In [ ]:
def grandparent(gp,c):
    '''
    This function takes two variables:
    gp: name of grandparent
    c: name of child
    It then queries the KB to find all the pairs for which there exist a parent p so that:
    gp is parent of p, p is parent of c
    The function returns an iteratable object containing all the matches
    '''
    # unknown parent variable
    p = mk.var()
    
    # `lall` it is a goal conjuction checker, it allows you to verify whether
    # all the goals in the sequence are True; equivalent to verify (A AND B)
    matches = mk.lall(parent(gp, p), parent(p, c))
    return matches

Let's query our kb using the new goal.

Because we want to just check for True/False we will use None instead of 0 , we will not need unknown variables, but we will add a `results_filter` function. This function checks whether the tuple of matches from our goal contains any element, if yes our query is True (= our goal is satisfied).


In [ ]:
results = mk.run(None, (), grandparent('John', 'Fran'), results_filter=lambda matches: len(tuple(matches))>0 )

print('Is John Fran`s grandparent:', results)

**Q2B: is Paul Mary's grandparent?**

In [ ]:
results = mk.run(None, (), grandparent('Paul', 'Mary'), results_filter=lambda matches: len(tuple(matches))>0 )

print('Is Paul Mary`s grandparent:', results)

### Task

**Q3: who is John's ancestor?**

Construct a goal similar to the one in Q2, and use the query approach from Q1. 

You can use the same rule defined for the pytholog example:
X is an ancestor of Z,  if _there exists_  Y such that X is the parent of Y _and_ Y is the grandparent of Z (`ancestor(X, Z) :- parent(X, Y),  grandparent(Y, Z)`).

In [ ]:
# write your code here

### Example - The research grant problem <a class="anchor" id="grant"></a>

Let's look at a more challenging example: the research grant problem.

There are 3 researchers: Aiko, Bilal, Catia, and Dinesh. 

Each has one research grant and one research subject. 

The available grants are: £25k, £30k, £35k and £40K. 

The available subjects are: Computer Science, Philosophy, Mathematics, and Arabic Studies. 

We know  the following clues:


1. The researcher who studies Computer Science gets a smaller grant than Aiko.
2. Aiko studies either Mathematics or Arabic Studies.
3. The researcher who studies Philosophy has a £5K bigger grant than Bilal.
4. Catia has a £10K bigger grant than Bilal.
5. Dinesh has a bigger grant than the researcher who studies Mathematics.

This type of puzzle can be solved by using pen&paper and _logic grids_ as the one below:


<img src="grant_grid.png" alt="grid" style="width: 200px;"/>


You can also access an online version of the grid: [grant_online_grid](https://jsingler.de/apps/logikloeser/?language=en#(at:s,items:!(!(Aiko,Bilal,Catia,Dinesh),!('25k','30k','35k','40k'),!('Comp.%20Sci.',Maths,Philosphy,Arabic)),ms:t,n:!(),nc:3,ni:4,p:!(),v:0)).

Using the facts above, we want to determine which researcher has which grant and studies which subject.

Let's solve the puzzle using logical programming!

In [ ]:
# Let's start by creating a dataclass object Researcher
# this object will allow us to store information for each researcher
from dataclasses import dataclass, field

@dataclass
class Researcher():
    # for each researcher we have a name, grant amount, and a research subject
    name : str = field(default_factory=mk.var)
    grant: int = field(default_factory=mk.var)
    subject: str = field(default_factory=mk.var)
        
# register Researcher class within the mk environment so we can call it later   
mk.unifiable(Researcher)          

In [ ]:
# We have 4 researchers in total, so lets create a suitable collection object where store all of them     
researchers = mk.vars(4)

all the clues, except for no2, ask to compare grants, so we are going to define a function to avoid repeating code.

In the function below we use membero, applyo and eq:
- `membero(item,coll`): states that item is a member of the collection coll
- `eq(x,y)`: express equality between x and y
- `applyo(f, terms, result_var)`: applies the function f to the terms listed and store the result in result_var

In [ ]:
from kanren.term import applyo

def compare_grants(r1,r2,callable):
    '''
    This function accepts two Research dataclass objects, and a callable function (like lambda x,y :  x>y)
    Then:
    - states that r1,r2 belong to the researchers collection (membero)
    - applies the callable functions to the grants and store the resulting comparison
    - checks that the comparison is True (eq) 
    '''
    g1 = r1.grant
    g2 = r2.grant
    comparison_result = mk.var()
    result = mk.lall(
        mk.membero(r1,researchers),
        mk.membero(r2,researchers),
        applyo(callable, (g1,g2), comparison_result),
        mk.eq(comparison_result, True)
    )
    return result

Each clue defined in the problem is a goal,  a relation that needs to be satisfied.

We write goal constructors for each of them.

In [ ]:
# Clue1 : The researcher who studies Computer Science gets a smaller grant than Aiko
# Callable comparison function lambda x,y: x<y

def clue1():
    r1 = Researcher(subject='Computer Science')
    r2 = Researcher(name='Aiko')
    result = compare_grants(r1,r2, lambda x,y: x<y)
    return result

In [ ]:
# Clue2 : Aiko studies either Mathematics or Arabic Studies

def clue2():
    g = Researcher(name='Aiko')
    s = g.subject
    result = mk.lall(
        mk.membero(g,researchers), 
        # lany verifies statements like (A OR B)
        mk.lany(mk.eq(s,'Mathematics'), mk.eq(s,'Arabic Studies'))
    )
    return result

In [ ]:
# Clue3: The researcher who studies Philosophy has a £5K bigger grant than Bilal
# Callable comparison function lambda x,y: x == y + 5

def clue3():
    r1 = Researcher(subject='Philosophy')
    r2 = Researcher(name='Bilal')
    result = compare_grants(r1,r2,lambda x,y: x == y + 5)
    return result

In [ ]:
# Clue4: Catia has a £10K bigger grant than Bilall
# Callable comparison function lambda x,y: x == y + 10

def clue4():
    r1 = Researcher(name='Catia')
    r2 = Researcher(name='Bilal')
    result = compare_grants(r1,r2,lambda x,y: x == y + 10)
    return result

In [ ]:
# Clue5: Dinesh has a bigger grant than the researcher who studies Mathematics
# Callable comparison function lambda x,y: x>y

def clue5():
    r1 = Researcher(name='Dinesh')
    r2 = Researcher(subject='Mathematics')
    result = compare_grants(r1,r2,lambda x,y : x > y)
    return result

We are now ready to put all the clues together to solve our puzzle.

Reading our clues, it is possible to understand what the 4 possible names and subjects of the reearchers are. However, the grant amount are not defined. In order to incorporate the information about the possible grant amounts (25, 30, 35, or 40) into our knowledge base, we need to explicitly state this information. For instance, we can write `eq(Researcher(grant=25), researchers[0])` to mean to to indicate that the researcher with a grant of 25 should be stored in the first slot of our `researchers` collection.

Finally, we need to specify that one researcher has subject of study = 'Arabic Studies'. this beacuase although we have used `Researcher(subject='Arabic Studies')` in the code, we have not stated its memebership within the `researchers` collection, as we have done with all the other combinations of names and subjects.

In [ ]:
# define a tuple containing all the goal.
goals = (
    mk.eq(Researcher(grant=25), researchers[0]),
    mk.eq(Researcher(grant=30), researchers[1]),
    mk.eq(Researcher(grant=35), researchers[2]),
    mk.eq(Researcher(grant=40), researchers[3]),
    clue1(),
    clue2(),
    clue3(),
    clue4(),
    clue5(),
    mk.membero(Researcher(subject='Arabic Studies'), researchers)
)

We are now ready to solve the puzzle...

In [ ]:
solution = mk.run(0, researchers, *goals)

solution

## Exercise - Logic puzzles  <a class="anchor" id="logic"></a>

For this exercise you are going to solve a similar problem to the one above. There two difficulties you can choose from.

### Easy
There are 3 kids named Aisha, Ben, and Celine, who are reading superhero comic books. The comic books they are reading are about Ironman, Spiderman, and Captain America. The ages of the kids are 6, 8, and 10 years old.

You know that:

- Aisha is reading Spiderman,
- Ben doesn't like Captain America
- The youngest kid is reading Spiderman
- The kid who reads Captain America is 8.

**Q: how old is the kid reading Iron Man?**

### Moderate
The [Zebra Puzzle](https://en.wikipedia.org/wiki/Zebra_Puzzle), originally published in Life International in 1962, is one of the oldest logic puzzles. 

You have the following 15 clues:

- There are five houses.
- The Englishman lives in the red house.
- The Spaniard owns the dog.
- Coffee is drunk in the green house.
- The Ukrainian drinks tea.
- The green house is immediately to the right of the ivory house.
- The Old Gold smoker owns snails.
- Kools are smoked in the yellow house.
- Milk is drunk in the middle house.
- The Norwegian lives in the first house.
- The man who smokes Chesterfields lives in the house next to the man with the fox.
- Kools are smoked in the house next to the house where the horse is kept.
- The Lucky Strike smoker drinks orange juice.
- The Japanese smokes Parliaments.
- The Norwegian lives next to the blue house.

You want answer the following questions:

**Q1: Who drinks water?**

**Q2: Who owns the zebra?**